Please note that I believe there might be some existing errors in coord extraction below. Indeed, part of the motivation of this exploration is to correct these errors and make it as easy as possible not to repeat them in user code.

In [ ]:
import numpy as np
import pydicom
import xarray as xr

In [ ]:
from pymedphys._data import download
from pymedphys._dicom import coords, dose

In [ ]:
dicom_dose_filepath = download.get_file_within_data_zip(
    "dicom_dose_test_data.zip", "RD.wedge.dcm"
)
ds = pydicom.dcmread(dicom_dose_filepath)

In [ ]:
dose = dose.dose_from_dataset(ds)
dose.shape

In [ ]:
extents = coords.xyz_axes_from_dataset(ds, coord_system="IEC FIXED")

In [ ]:
dose_arr = xr.DataArray(
    dose,
    coords = {
        'x': extents[0],
        'y': extents[1],
        'z': extents[2],
    },
    dims=['y', 'z', 'x'],
    name="dose",
    attrs={
        "units":"Gy",
        "long_name": "Dose to water from wedged field."
    }
)

In [ ]:
dose_arr.x

In [ ]:
dose_arr.y

In [ ]:
dose_arr.z

In [ ]:
# Need method="nearest" if exact coordinate isn't suppled to sel():
try:
    dose_arr.sel(x=0)
except KeyError:
    print("KeyError")

In [ ]:
dose_arr.sel(x=0, method="nearest")

In [ ]:
# Very easily plot dose slice, already labelled with colorbar and units
dose_arr.sel(y=0, method="nearest").plot(cmap="jet", figsize=(8,6), aspect="equal") #Notice we asked for y=0, but plotting y=211 (closest slice)

In [ ]:
# Lovely inbuilt interpolation too!

x_new = np.linspace(dose_arr.x[0], dose_arr.x[-1], dose_arr.sizes["x"]*4) # Make 4x finer
y_new = np.linspace(dose_arr.y[0], dose_arr.y[-1], dose_arr.sizes["y"]*4) # Make 4x finer
z_new = np.linspace(dose_arr.z[0], dose_arr.z[-1], dose_arr.sizes["z"]*4) # Make 4x finer

dose_arr_interp = dose_arr.interp(x=x_new, y=y_new, z=z_new)
dose_arr_interp.sel(y=0, method="nearest").plot(cmap="jet", figsize=(8,6), aspect="equal")

In [ ]:
# Can perhaps add coordinate system and patient orientation (where applicable) as attrs?

dose_arr.attrs["coord_system"]="IEC FIXED"
dose_arr.attrs["orientation"]="HFS"
dose_arr # see bottom

In [ ]:
# And then use swap_dims() and negations where required to change coord_system?
 
def change_xarray_coord_system_from_fixed_to_dicom(dose_arr): # generalise in lib but just demo for now
    
    new_dose_arr = dose_arr.rename(
        {
            "z": "y",
            "y": "z"
        }
    )

    new_dose_arr["y"] = -new_dose_arr["y"]
    new_dose_arr.attrs["coord_system"] = "DICOM"

    return new_dose_arr

In [ ]:
dose_arr.coords

In [ ]:
dose_arr_dicomCS = change_xarray_coord_system_from_fixed_to_dicom(dose_arr)

dose_arr_dicomCS.coords # Note swapping of y and z (DICOM at HFS) and negative y values (positive z before)

In [ ]:
dose_arr_dicomCS.attrs["coord_system"]

In [ ]:
# Notice this is the same slice as before (since we chose z=0), but labelled according to the new (DICOM) coord system. It is flipped vertically because the 'y' in DICOM is negative relative to 'z" in IEC FIXED. The plot function sorts the ordinate labels to be increasing from the bottom, and flips the dose data accordingly. Not sure where this is happening but should be easy to either disable or workaround (I think people are used to visualising in a FIXED system, so I believe it is better to simply reverse labels). 
dose_arr_dicomCS.sel(z=0, method="nearest").plot(cmap="jet", figsize=(8,6), aspect="equal") 